In [ ]:
#Kütüphaneleri Yükleme
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, BayesianRidge, Lars, LassoLars
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor, AdaBoostRegressor, HistGradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import HuberRegressor, OrthogonalMatchingPursuit
from sklearn.dummy import DummyRegressor
from sklearn.experimental import enable_hist_gradient_boosting
from catboost import CatBoostRegressor
import lightgbm as lgb
import xgboost as xgb
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import GridSearchCV

2024-09-15 14:24:09.180518: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-15 14:24:09.354570: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-15 14:24:09.418962: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-15 14:24:09.439359: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-15 14:24:09.567188: I tensorflow/core/platform/cpu_feature_guar

In [ ]:
#Verileri Yükleme
raw_train = pd.read_csv("train.csv")
raw_test = pd.read_csv("test_x.csv")
raw_submission = pd.read_csv("sample_submission.csv")

/tmp/ipykernel_67435/2427738919.py:2: DtypeWarning: Columns (13,15,19,30,32,37,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_train = pd.read_csv("train.csv")


In [ ]:
new_train = raw_train.drop(columns=["id"])

In [ ]:
concated_df = pd.concat([raw_test, raw_train], ignore_index=False)

In [ ]:
concated_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 76174 entries, 0 to 65124
Data columns (total 44 columns):
 #   Column                                                     Non-Null Count  Dtype  
---  ------                                                     --------------  -----  
 0   Basvuru Yili                                               76174 non-null  int64  
 1   Cinsiyet                                                   76005 non-null  object 
 2   Dogum Tarihi                                               75997 non-null  object 
 3   Dogum Yeri                                                 75383 non-null  object 
 4   Ikametgah Sehri                                            74137 non-null  object 
 5   Universite Adi                                             76042 non-null  object 
 6   Universite Turu                                            75919 non-null  object 
 7   Burslu ise Burs Yuzdesi                                    22440 non-null  float64
 8   Burs Aliyor

In [ ]:
df_shuffled = concated_df.sample(frac=1).reset_index(drop=True)

In [ ]:
df_shuffled.drop(columns=["id"],inplace=True)

In [ ]:
missing_rate = df_shuffled.isnull().sum() / len(df_shuffled) * 100
print(missing_rate[missing_rate > 0].sort_values(ascending=False))

Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite        99.449944
Lise Bolum Diger                                             97.996692
Lise Adi Diger                                               93.786594
Uye Oldugunuz Kulubun Ismi                                   78.425710
Burslu ise Burs Yuzdesi                                      70.541130
Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?    66.329719
Hangi STK'nin Uyesisiniz?                                    65.006433
Daha Once Baska Bir Universiteden Mezun Olmus                63.530864
Ingilizce Seviyeniz?                                         62.464883
Baska Kurumdan Aldigi Burs Miktari                           57.975162
Burs Aldigi Baska Kurum                                      57.732297
Stk Projesine Katildiniz Mi?                                 51.667236
Anne Sektor                                                  45.339617
Spor Dalindaki Rolunuz Nedir?                                42.370100
Baba S

In [ ]:
test_null_columns = raw_test.columns[raw_test.isnull().any()]
print(test_null_columns)

Index(['Burslu ise Burs Yuzdesi',
       'Daha Once Baska Bir Universiteden Mezun Olmus', 'Lise Adi Diger',
       'Lise Bolum Diger', 'Uye Oldugunuz Kulubun Ismi',
       'Stk Projesine Katildiniz Mi?', 'Ingilizce Seviyeniz?',
       'Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite'],
      dtype='object')


In [ ]:
cikarilcak_kolonlar = ["Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite",
                        "Lise Bolum Diger",
                        "Lise Adi Diger",
                        "Hangi STK'nin Uyesisiniz?",
                        "Uye Oldugunuz Kulubun Ismi",
                        "Ikametgah Sehri",
                        "Kardes Sayisi",
                        "Lise Sehir",
                        "Lise Bolumu",
                        "Dogum Yeri",
                        "Lise Adi",
                        "Dogum Tarihi",
                        "Baska Kurumdan Aldigi Burs Miktari",
                        "Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?"
]

In [ ]:
cikarilmis_train = df_shuffled.drop(columns=cikarilcak_kolonlar)

In [ ]:
cikarilmis_test = raw_test.drop(columns=cikarilcak_kolonlar)

In [ ]:
cinsiyet_mapping = {'Erkek': 1, 'ERKEK': 1, 'Kadın': 2, 'Belirtmek istemiyorum': 0}
uni_mapping = {'Devlet': 1, 'DEVLET': 1, 'Özel': 0, 'ÖZEL': 0}
burs_mapping = {'evet': 1, 'Evet': 1, 'EVET': 1,'Hayır': 0, 'hayır': 0}
uni_sinif_mapping = {'1': 1, '2': 2, '3': 3,'4': 4, '5': 5,'6':6,'Yüksek Lisans':7,'tez':7,'Mezun':0,"Hazırlık":0,"hazırlık":0,"0":0}
uni_not_mapping = {'ORTALAMA BULUNMUYOR': 0, 'Not ortalaması yok': 0, 'Ortalama bulunmuyor': 0,'Hazırlığım': 0,
                   '0 - 1.79': 1,'1.00 - 2.50':2,'1.80 - 2.49':2,'2.50 ve altı':2,'2.00 - 2.50':2,"3.00-2.50":3,
                   "2.50 - 3.00":3,"2.50 - 2.99":3,'2.50 -3.00':3,'3.00 - 3.50':4,'3.50-3':4,'3.00 - 3.49':4,'3.00 - 4.00':4,'3.50 - 4.00':5,'4-3.5':5,'4.0-3.5':5}
mezun_mapping = {'Evet': 1, 'Hayır': 0}
lise_turu_mapping = {'İmam Hatip Lisesi': 1, 'Diğer': 0, 'Devlet': 0,'Düz lise': 0,
                   'Düz Lise': 0,'Meslek lisesi':1,'Meslek':1,'Meslek Lisesi':1,'Özel':2,"Özel Lisesi":2,
                   "Özel lisesi":2,"Özel Lise":1,'Anadolu Lisesi':2,'Anadolu lisesi':2,'Fen lisesi':3,'Fen Lisesi':3}
lise_not_mapping = {
    '75 - 100': 5,
    '84-70': 4,
    '100-85': 5,
    '4.00-3.50': 5,
    '50 - 75': 3,
    '3.00 - 4.00': 4,
    '3.50-3.00': 4,
    '3.50-3': 4,
    '69-55': 3,
    '3.00-2.50': 3,
    '50 - 74': 3,
    '2.50 ve altı': 1,
    '54-45': 2,
    '25 - 50': 1,
    'Not ortalaması yok': 0,
    '44-0': 0,
    '0 - 25': 0,
    '25 - 49': 1,
    '0 - 24': 0
}
burslu_mapping = {'Evet': 1, 'Hayır': 0}
anne_egitim_mapping = {
    'İlkokul': 1,
    'İlkokul Mezunu': 1,
    'İLKOKUL MEZUNU': 1,
    'Eğitimi yok': 0,
    'EĞİTİM YOK': 0,
    'Eğitim Yok': 0,
    'Ortaokul': 2,
    'Ortaokul Mezunu': 2,
    'ORTAOKUL MEZUNU': 2,
    'Lise': 3,
    'LİSE': 3,
    'Lise Mezunu': 3,
    'Üniversite': 4,
    'ÜNİVERSİTE': 4,
    'Üniversite Mezunu': 4,
    'Yüksek Lisans': 5,
    'YÜKSEK LİSANS': 5,
    'Yüksek Lisans / Doktora': 6,
    'Yüksek Lisans / Doktara': 6,
    'Doktora': 6,
    'DOKTORA': 6
}
anne_calisma_mapping = {'Evet': 1, 'Hayır': 0,'Emekli':2}
anne_sektor_mapping = {'0': 0, '-': 0,'Özel Sektör':1,'ÖZEL SEKTÖR':1,'Diğer':1,'DİĞER':1,'Kamu':2,'KAMU':2}
baba_egitim_mapping = {
    'İlkokul': 1,
    'İlkokul Mezunu': 1,
    'İLKOKUL MEZUNU': 1,
    'Eğitimi yok': 0,
    'EĞİTİM YOK': 0,
    'Eğitim Yok': 0,
    'Ortaokul': 2,
    'Ortaokul Mezunu': 2,
    'ORTAOKUL MEZUNU': 2,
    'Lise': 3,
    'LİSE': 3,
    'Lise Mezunu': 3,
    'Üniversite': 4,
    'ÜNİVERSİTE': 4,
    'Üniversite Mezunu': 4,
    'Yüksek Lisans': 5,
    'YÜKSEK LİSANS': 5,
    'Yüksek Lisans / Doktora': 6,
    'Yüksek Lisans / Doktara': 6,
    'Doktora': 6,
    'DOKTORA': 6,
    '0':0
}
baba_calisma_mapping = {'Evet': 1, 'Hayır': 0,'Emekli':2}
baba_sektor_mapping = {'0': 0, '-': 0,'Özel Sektör':1,'ÖZEL SEKTÖR':1,'Diğer':1,'DİĞER':1,'Kamu':2,'KAMU':2}
girisim_kulub_mapping = {'Hayır': 0, 'Evet': 1}
spor_mapping = {'Hayır': 0, 'Evet': 1}
spor_rol_mapping = {'0': 0, '-': 0,'Diğer':0,'DİĞER':0,'Bireysel':1,'Takım Oyuncusu':2,'Lider/Kaptan':3,'KAPTAN / LİDER':3,'Kaptan':3}
stk_mapping = {'Hayır': 0, 'Evet': 1}
stk2_mapping = {'Hayır': 0, 'Evet': 1}
girisim_mapping = {'Hayır': 0, 'Evet': 1}
ing_mapping = {'Hayır': 0, 'Evet': 1}
ing_seviye_mapping = {'0': 0, 'Başlangıç': 1,'Orta':2,'İleri':3}

In [ ]:
cikarilmis_train['Universite Kacinci Sinif'] = cikarilmis_train['Universite Kacinci Sinif'].map(uni_sinif_mapping)
cikarilmis_train['Burs Aliyor mu?'] = cikarilmis_train['Burs Aliyor mu?'].map(burs_mapping)
cikarilmis_train['Cinsiyet'] = cikarilmis_train['Cinsiyet'].map(cinsiyet_mapping)
cikarilmis_train['Universite Turu'] = cikarilmis_train['Universite Turu'].map(uni_mapping)
cikarilmis_train['Universite Not Ortalamasi'] = cikarilmis_train['Universite Not Ortalamasi'].map(uni_not_mapping)
cikarilmis_train['Daha Once Baska Bir Universiteden Mezun Olmus'] = cikarilmis_train['Daha Once Baska Bir Universiteden Mezun Olmus'].map(mezun_mapping)
cikarilmis_train['Lise Turu'] = cikarilmis_train['Lise Turu'].map(lise_turu_mapping)
cikarilmis_train['Lise Mezuniyet Notu'] = cikarilmis_train['Lise Mezuniyet Notu'].map(lise_not_mapping)
cikarilmis_train['Baska Bir Kurumdan Burs Aliyor mu?'] = cikarilmis_train['Baska Bir Kurumdan Burs Aliyor mu?'].map(burslu_mapping)
cikarilmis_train["Anne Egitim Durumu"] = cikarilmis_train["Anne Egitim Durumu"].map(anne_egitim_mapping)
cikarilmis_train['Anne Calisma Durumu'] = cikarilmis_train['Anne Calisma Durumu'].map(anne_calisma_mapping)
cikarilmis_train['Anne Sektor'] = cikarilmis_train['Anne Sektor'].map(anne_sektor_mapping)
cikarilmis_train["Baba Egitim Durumu"] = cikarilmis_train["Baba Egitim Durumu"].map(baba_egitim_mapping)
cikarilmis_train['Baba Calisma Durumu'] = cikarilmis_train['Baba Calisma Durumu'].map(baba_calisma_mapping)
cikarilmis_train['Baba Sektor'] = cikarilmis_train['Baba Sektor'].map(baba_sektor_mapping)
cikarilmis_train['Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?'] = cikarilmis_train['Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?'].map(girisim_kulub_mapping)
cikarilmis_train['Profesyonel Bir Spor Daliyla Mesgul musunuz?'] = cikarilmis_train['Profesyonel Bir Spor Daliyla Mesgul musunuz?'].map(spor_mapping)
cikarilmis_train['Spor Dalindaki Rolunuz Nedir?'] = cikarilmis_train['Spor Dalindaki Rolunuz Nedir?'].map(spor_rol_mapping)
cikarilmis_train['Aktif olarak bir STK üyesi misiniz?'] = cikarilmis_train['Aktif olarak bir STK üyesi misiniz?'].map(stk_mapping)
cikarilmis_train['Stk Projesine Katildiniz Mi?'] = cikarilmis_train['Stk Projesine Katildiniz Mi?'].map(stk2_mapping)
cikarilmis_train['Girisimcilikle Ilgili Deneyiminiz Var Mi?'] = cikarilmis_train['Girisimcilikle Ilgili Deneyiminiz Var Mi?'].map(girisim_mapping)
cikarilmis_train['Ingilizce Biliyor musunuz?'] = cikarilmis_train['Ingilizce Biliyor musunuz?'].map(ing_mapping)
cikarilmis_train['Ingilizce Seviyeniz?'] = cikarilmis_train['Ingilizce Seviyeniz?'].map(ing_seviye_mapping)

In [ ]:
cikarilmis_test['Universite Kacinci Sinif'] = cikarilmis_test['Universite Kacinci Sinif'].map(uni_sinif_mapping)
cikarilmis_test['Burs Aliyor mu?'] = cikarilmis_test['Burs Aliyor mu?'].map(burs_mapping)
cikarilmis_test['Cinsiyet'] = cikarilmis_test['Cinsiyet'].map(cinsiyet_mapping)
cikarilmis_test['Universite Turu'] = cikarilmis_test['Universite Turu'].map(uni_mapping)
cikarilmis_test['Universite Not Ortalamasi'] = cikarilmis_test['Universite Not Ortalamasi'].map(uni_not_mapping)
cikarilmis_test['Lise Turu'] = cikarilmis_test['Lise Turu'].map(lise_turu_mapping)
cikarilmis_test['Lise Mezuniyet Notu'] = cikarilmis_test['Lise Mezuniyet Notu'].map(lise_not_mapping)
cikarilmis_test['Baska Bir Kurumdan Burs Aliyor mu?'] = cikarilmis_test['Baska Bir Kurumdan Burs Aliyor mu?'].map(burslu_mapping)
cikarilmis_test["Anne Egitim Durumu"] = cikarilmis_test["Anne Egitim Durumu"].map(anne_egitim_mapping)
cikarilmis_test['Anne Calisma Durumu'] = cikarilmis_test['Anne Calisma Durumu'].map(anne_calisma_mapping)
cikarilmis_test['Anne Sektor'] = cikarilmis_test['Anne Sektor'].map(anne_sektor_mapping)
cikarilmis_test["Baba Egitim Durumu"] = cikarilmis_test["Baba Egitim Durumu"].map(baba_egitim_mapping)
cikarilmis_test['Baba Calisma Durumu'] = cikarilmis_test['Baba Calisma Durumu'].map(baba_calisma_mapping)
cikarilmis_test['Baba Sektor'] = cikarilmis_test['Baba Sektor'].map(baba_sektor_mapping)
cikarilmis_test['Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?'] = cikarilmis_test['Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?'].map(girisim_kulub_mapping)
cikarilmis_test['Profesyonel Bir Spor Daliyla Mesgul musunuz?'] = cikarilmis_test['Profesyonel Bir Spor Daliyla Mesgul musunuz?'].map(spor_mapping)
cikarilmis_test['Spor Dalindaki Rolunuz Nedir?'] = cikarilmis_test['Spor Dalindaki Rolunuz Nedir?'].map(spor_rol_mapping)
cikarilmis_test['Aktif olarak bir STK üyesi misiniz?'] = cikarilmis_test['Aktif olarak bir STK üyesi misiniz?'].map(stk_mapping)
cikarilmis_test['Girisimcilikle Ilgili Deneyiminiz Var Mi?'] = cikarilmis_test['Girisimcilikle Ilgili Deneyiminiz Var Mi?'].map(girisim_mapping)
cikarilmis_test['Ingilizce Biliyor musunuz?'] = cikarilmis_test['Ingilizce Biliyor musunuz?'].map(ing_mapping)

In [ ]:
missing_values_count = cikarilmis_train.isna().sum()

# Eksik veri bulunan kolonları listele
missing_columns = missing_values_count[missing_values_count > 0].index.tolist()

missing_columns

['Cinsiyet',
 'Universite Adi',
 'Universite Turu',
 'Burslu ise Burs Yuzdesi',
 'Bölüm',
 'Universite Kacinci Sinif',
 'Universite Not Ortalamasi',
 'Daha Once Baska Bir Universiteden Mezun Olmus',
 'Lise Turu',
 'Lise Mezuniyet Notu',
 'Baska Bir Kurumdan Burs Aliyor mu?',
 'Burs Aldigi Baska Kurum',
 'Anne Egitim Durumu',
 'Anne Calisma Durumu',
 'Anne Sektor',
 'Baba Egitim Durumu',
 'Baba Calisma Durumu',
 'Baba Sektor',
 'Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?',
 'Profesyonel Bir Spor Daliyla Mesgul musunuz?',
 'Spor Dalindaki Rolunuz Nedir?',
 'Aktif olarak bir STK üyesi misiniz?',
 'Stk Projesine Katildiniz Mi?',
 'Girisimcilikle Ilgili Deneyiminiz Var Mi?',
 'Ingilizce Biliyor musunuz?',
 'Ingilizce Seviyeniz?',
 'Degerlendirme Puani']

In [ ]:
missing_values_count = cikarilmis_test.isna().sum()

# Eksik veri bulunan kolonları listele
missing_columns = missing_values_count[missing_values_count > 0].index.tolist()

missing_columns

['Burslu ise Burs Yuzdesi',
 'Daha Once Baska Bir Universiteden Mezun Olmus',
 'Spor Dalindaki Rolunuz Nedir?',
 'Stk Projesine Katildiniz Mi?',
 'Ingilizce Seviyeniz?']

In [ ]:
cikarilmis_train['Cinsiyet'] = cikarilmis_train['Cinsiyet'].fillna(0)
cikarilmis_train['Universite Adi'] = cikarilmis_train['Universite Adi'].fillna(0)
cikarilmis_train['Burslu ise Burs Yuzdesi'] = cikarilmis_train['Burslu ise Burs Yuzdesi'].fillna(0)
cikarilmis_train['Bölüm'] = cikarilmis_train['Bölüm'].fillna(0)
cikarilmis_train['Universite Kacinci Sinif'] = cikarilmis_train['Universite Kacinci Sinif'].fillna(0)
cikarilmis_train["Universite Turu"] = cikarilmis_train["Universite Turu"].fillna(2)
cikarilmis_train['Universite Not Ortalamasi'] = cikarilmis_train['Universite Not Ortalamasi'].fillna(0)
cikarilmis_train['Baska Bir Kurumdan Burs Aliyor mu?'] = cikarilmis_train['Baska Bir Kurumdan Burs Aliyor mu?'].fillna(0)
cikarilmis_train['Anne Egitim Durumu'] = cikarilmis_train['Anne Egitim Durumu'].fillna(0)
cikarilmis_train['Anne Sektor'] = cikarilmis_train['Anne Sektor'].fillna(0)
cikarilmis_train['Baba Egitim Durumu'] = cikarilmis_train['Baba Egitim Durumu'].fillna(0)
cikarilmis_train['Baba Sektor'] = cikarilmis_train['Baba Sektor'].fillna(0)
cikarilmis_train['Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?'] = cikarilmis_train['Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?'].fillna(0)
cikarilmis_train['Profesyonel Bir Spor Daliyla Mesgul musunuz?'] = cikarilmis_train['Profesyonel Bir Spor Daliyla Mesgul musunuz?'].fillna(0)
cikarilmis_train['Spor Dalindaki Rolunuz Nedir?'] = cikarilmis_train['Spor Dalindaki Rolunuz Nedir?'].fillna(0)
cikarilmis_train['Stk Projesine Katildiniz Mi?'] = cikarilmis_train['Stk Projesine Katildiniz Mi?'].fillna(2)
cikarilmis_train['Girisimcilikle Ilgili Deneyiminiz Var Mi?'] = cikarilmis_train['Girisimcilikle Ilgili Deneyiminiz Var Mi?'].fillna(0)
cikarilmis_train['Ingilizce Biliyor musunuz?'] = cikarilmis_train['Ingilizce Biliyor musunuz?'].fillna(0)
cikarilmis_train['Ingilizce Seviyeniz?'] = cikarilmis_train['Ingilizce Seviyeniz?'].fillna(0)
cikarilmis_train['Daha Once Baska Bir Universiteden Mezun Olmus'] = cikarilmis_train['Daha Once Baska Bir Universiteden Mezun Olmus'].fillna(2)
cikarilmis_train['Lise Turu'] = cikarilmis_train['Lise Turu'].fillna(0)
cikarilmis_train['Lise Mezuniyet Notu']= cikarilmis_train['Lise Mezuniyet Notu'].fillna(0)
cikarilmis_train['Anne Calisma Durumu']= cikarilmis_train['Anne Calisma Durumu'].fillna(0)
cikarilmis_train['Baba Calisma Durumu']= cikarilmis_train['Baba Calisma Durumu'].fillna(0)
cikarilmis_train['Aktif olarak bir STK üyesi misiniz?']= cikarilmis_train['Aktif olarak bir STK üyesi misiniz?'].fillna(0)

In [ ]:
def ran_impute(df):
    df_encoded = df.copy()
    for col in df_encoded.select_dtypes(include='object').columns:
        df_encoded[col] = df_encoded[col].astype('category').cat.codes
    ran_imputer = IterativeImputer(estimator=RandomForestRegressor(n_estimators=30,random_state=42,n_jobs=-1),max_iter=15,random_state=42)
    df_imputed = pd.DataFrame(ran_imputer.fit_transform(df_encoded), columns=df_encoded.columns)
    for col in df.select_dtypes(include='object').columns:
        df_imputed[col] = df_imputed[col].round().astype(int).map(
            dict(enumerate(df[col].astype('category').cat.categories)))
    return df_imputed

In [ ]:
def ran_impute_test(df_test,df_train):
    df_test_encoded = df_test.copy()
    df_train_encoded = df_train.copy()
    for col in df_test_encoded.select_dtypes(include='object').columns:
        df_test_encoded[col] = df_test_encoded[col].astype('category').cat.codes
    for col in df_train_encoded.select_dtypes(include='object').columns:
        df_train_encoded[col] = df_train_encoded[col].astype('category').cat.codes
    ran_imputer = IterativeImputer(estimator=RandomForestRegressor(n_estimators=25,random_state=42,n_jobs=-1),max_iter=14,random_state=42)
    ran_imputer.fit(df_train_encoded)
    df_test_imputed = pd.DataFrame(ran_imputer.transform(df_test_encoded), columns=df_test_encoded.columns)
    for col in df_test.select_dtypes(include='object').columns:
        df_test_imputed[col] = df_test_imputed[col].round().astype(int).map(
            dict(enumerate(df_test[col].astype('category').cat.categories)))
    return df_test_imputed

In [ ]:
test_df_idsiz = cikarilmis_test.drop(columns=["id"])

In [ ]:
train_df_degersiz = cikarilmis_train.drop(columns=["Degerlendirme Puani"])

In [ ]:
df_test_imputed = ran_impute_test(test_df_idsiz,train_df_degersiz)

In [ ]:
df_imputed2 = ran_impute(cikarilmis_train)

In [ ]:
df_test_imputed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11049 entries, 0 to 11048
Data columns (total 28 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   Basvuru Yili                                          11049 non-null  float64
 1   Cinsiyet                                              11049 non-null  float64
 2   Universite Adi                                        11049 non-null  object 
 3   Universite Turu                                       11049 non-null  float64
 4   Burslu ise Burs Yuzdesi                               11049 non-null  float64
 5   Burs Aliyor mu?                                       11049 non-null  float64
 6   Bölüm                                                 11049 non-null  object 
 7   Universite Kacinci Sinif                              11049 non-null  float64
 8   Universite Not Ortalamasi                             11

In [ ]:
sayisal_kolon = df_imputed2.select_dtypes(include=['float64', 'int64']).columns

kategorik_kolon = df_imputed2.select_dtypes(include=['object']).columns

In [ ]:
sayisal_kolon_test = df_test_imputed.select_dtypes(include=['float64', 'int64']).columns

kategorik_kolon_test = df_test_imputed.select_dtypes(include=['object']).columns

In [ ]:
for i in kategorik_kolon_test:
    df_test_imputed[i] = df_test_imputed[i].astype(str)

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoders = {}

for i in kategorik_kolon:
    le = LabelEncoder()
    df_imputed2[i] = le.fit_transform(df_imputed2[i])
    label_encoders[i] = le

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoders = {}

for i in kategorik_kolon:
    le = LabelEncoder()
    df_test_imputed[i] = le.fit_transform(df_test_imputed[i])
    label_encoders[i] = le

In [ ]:
df_imputed2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76174 entries, 0 to 76173
Data columns (total 29 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   Basvuru Yili                                          76174 non-null  float64
 1   Cinsiyet                                              76174 non-null  float64
 2   Universite Adi                                        76174 non-null  int64  
 3   Universite Turu                                       76174 non-null  float64
 4   Burslu ise Burs Yuzdesi                               76174 non-null  float64
 5   Burs Aliyor mu?                                       76174 non-null  float64
 6   Bölüm                                                 76174 non-null  int64  
 7   Universite Kacinci Sinif                              76174 non-null  float64
 8   Universite Not Ortalamasi                             76

In [ ]:
df_test_imputed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11049 entries, 0 to 11048
Data columns (total 28 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   Basvuru Yili                                          11049 non-null  float64
 1   Cinsiyet                                              11049 non-null  float64
 2   Universite Adi                                        11049 non-null  int64  
 3   Universite Turu                                       11049 non-null  float64
 4   Burslu ise Burs Yuzdesi                               11049 non-null  float64
 5   Burs Aliyor mu?                                       11049 non-null  float64
 6   Bölüm                                                 11049 non-null  int64  
 7   Universite Kacinci Sinif                              11049 non-null  float64
 8   Universite Not Ortalamasi                             11

In [ ]:
df_test_imputed['id'] = cikarilmis_test['id']

In [ ]:
x_test = df_test_imputed.drop(columns=["id"])

In [ ]:
x = df_imputed2.drop(columns=["Degerlendirme Puani"])
y = df_imputed2["Degerlendirme Puani"]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
linear_model = LinearRegression()
linear_model.fit(x_train, y_train)
y_pred = linear_model.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print("MSE:", mse)
print("RMSE:", rmse)

MSE: 65.461281767188
RMSE: 8.09081465411166


In [ ]:
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'Elastic Net': ElasticNet(),
    'Bayesian Ridge': BayesianRidge(),
    'Least Angle Regression': Lars(),
    'Lasso Least Angle Regression': LassoLars(),
    'Gradient Boosting Regressor': GradientBoostingRegressor(),
    'Random Forest Regressor': RandomForestRegressor(),
    'Extra Trees Regressor': ExtraTreesRegressor(),
    'AdaBoost Regressor': AdaBoostRegressor(),
    'Decision Tree Regressor': DecisionTreeRegressor(),
    'K Neighbors Regressor': KNeighborsRegressor(),
    'Huber Regressor': HuberRegressor(),
    'Orthogonal Matching Pursuit': OrthogonalMatchingPursuit(),
    'Dummy Regressor': DummyRegressor(strategy='mean'),
    'Extreme Gradient Boosting': xgb.XGBRegressor(),
    'Light Gradient Boosting Machine': lgb.LGBMRegressor(),
    'CatBoost Regressor': CatBoostRegressor(learning_rate=0.1, depth=6, iterations=500, verbose=0),
    'Hist Gradient Boosting Regressor': HistGradientBoostingRegressor()
}

results = {}

for name, model in models.items():
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    results[name] = rmse
    print(f'{name}: RMSE = {rmse}')

results_df = pd.DataFrame(list(results.items()), columns=['Model', 'RMSE'])
print(results_df.sort_values(by='RMSE'))

Linear Regression: RMSE = 8.040030105986986
Ridge Regression: RMSE = 8.040032571899589
Lasso Regression: RMSE = 9.490506146213296
Elastic Net: RMSE = 11.807791821083535
Bayesian Ridge: RMSE = 8.040038820680254
Least Angle Regression: RMSE = 8.104736899818352
Lasso Least Angle Regression: RMSE = 9.490483306213738
Gradient Boosting Regressor: RMSE = 5.784090271298295
Random Forest Regressor: RMSE = 5.319493743786908
Extra Trees Regressor: RMSE = 5.539897588109868
AdaBoost Regressor: RMSE = 10.072490401342343
Decision Tree Regressor: RMSE = 7.369192435181964
K Neighbors Regressor: RMSE = 16.154388218267986


/home/yasarkagant/miniconda3/envs/tf216/lib/python3.11/site-packages/sklearn/linear_model/_huber.py:342: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Huber Regressor: RMSE = 11.004043440754259
Orthogonal Matching Pursuit: RMSE = 17.534014067415082
Dummy Regressor: RMSE = 17.545128544216812
Extreme Gradient Boosting: RMSE = 5.1547393598207325
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005495 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 878
[LightGBM] [Info] Number of data points in the train set: 68556, number of used features: 28
[LightGBM] [Info] Start training from score 31.948403
Light Gradient Boosting Machine: RMSE = 5.234925336603928
CatBoost Regressor: RMSE = 5.039656439478337
Hist Gradient Boosting Regressor: RMSE = 5.247337935330382
                               Model       RMSE
18                CatBoost Regressor   5.039656
16         Extreme Gradient Boosting   5.154739
17   Light Gra

In [ ]:
param_grid = {
    'iterations': [500, 1000],
    'depth': [6, 8, 10],
    'learning_rate': [0.01, 0.05, 0.1],
    'l2_leaf_reg': [3, 5, 10],
    'bagging_temperature': [0.5, 1.0]
}
model = CatBoostRegressor()
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='neg_root_mean_squared_error', verbose=2)
grid_search.fit(x_train, y_train)

print("Best Parameters:", grid_search.best_params_)

Görüntülenen çıkış son 5000 satıra kısaltıldı.
5:	learn: 11.3154175	total: 54.8ms	remaining: 9.09s
6:	learn: 10.6496756	total: 66.6ms	remaining: 9.44s
7:	learn: 10.0551424	total: 89.4ms	remaining: 11.1s
8:	learn: 9.5330264	total: 97.8ms	remaining: 10.8s
9:	learn: 9.0575612	total: 108ms	remaining: 10.6s
10:	learn: 8.6434493	total: 118ms	remaining: 10.6s
11:	learn: 8.3081452	total: 129ms	remaining: 10.7s
12:	learn: 8.0088408	total: 140ms	remaining: 10.6s
13:	learn: 7.7373554	total: 148ms	remaining: 10.4s
14:	learn: 7.4892395	total: 157ms	remaining: 10.3s
15:	learn: 7.2681053	total: 167ms	remaining: 10.3s
16:	learn: 7.0868246	total: 178ms	remaining: 10.3s
17:	learn: 6.9158170	total: 188ms	remaining: 10.3s
18:	learn: 6.7693459	total: 198ms	remaining: 10.2s
19:	learn: 6.6371956	total: 207ms	remaining: 10.2s
20:	learn: 6.5159923	total: 216ms	remaining: 10.1s
21:	learn: 6.4119808	total: 225ms	remaining: 9.98s
22:	learn: 6.3166939	total: 232ms	remaining: 9.87s
23:	learn: 6.2370270	total: 240ms

In [ ]:
model =CatBoostRegressor(learning_rate=0.1, depth=6, iterations=1000, bagging_temperature =  0.5,l2_leaf_reg= 3)
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'Cat Boost Regressor :  RMSE = {rmse}')

0:	learn: 16.1467649	total: 8.85ms	remaining: 8.84s
1:	learn: 14.9750838	total: 13.3ms	remaining: 6.66s
2:	learn: 13.9451897	total: 20.6ms	remaining: 6.86s
3:	learn: 13.0469213	total: 25ms	remaining: 6.23s
4:	learn: 12.2671358	total: 28.1ms	remaining: 5.59s
5:	learn: 11.5772112	total: 31.7ms	remaining: 5.25s
6:	learn: 10.9816770	total: 35.3ms	remaining: 5.01s
7:	learn: 10.4670385	total: 39ms	remaining: 4.84s
8:	learn: 10.0072849	total: 41.6ms	remaining: 4.58s
9:	learn: 9.5939447	total: 45.4ms	remaining: 4.5s
10:	learn: 9.2345993	total: 47.8ms	remaining: 4.3s
11:	learn: 8.9120163	total: 51.7ms	remaining: 4.26s
12:	learn: 8.6196643	total: 54.3ms	remaining: 4.12s
13:	learn: 8.3771157	total: 57.8ms	remaining: 4.07s
14:	learn: 8.1424289	total: 61.2ms	remaining: 4.02s
15:	learn: 7.9452134	total: 65.6ms	remaining: 4.04s
16:	learn: 7.7325969	total: 68.5ms	remaining: 3.96s
17:	learn: 7.5760604	total: 72.2ms	remaining: 3.94s
18:	learn: 7.3994914	total: 74.6ms	remaining: 3.85s
19:	learn: 7.263966

In [ ]:
model_ =CatBoostRegressor(learning_rate=0.1, depth=6, iterations=1000, bagging_temperature =  0.5,l2_leaf_reg= 3)
model_.fit(x, y)
y_pred = model.predict(x_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'Cat Boost Regressor :  RMSE = {rmse}')

0:	learn: 16.1702266	total: 9.51ms	remaining: 9.5s
1:	learn: 14.9961856	total: 18.2ms	remaining: 9.07s
2:	learn: 13.9651260	total: 23.8ms	remaining: 7.92s
3:	learn: 13.0664293	total: 28.3ms	remaining: 7.04s
4:	learn: 12.2859141	total: 33.6ms	remaining: 6.68s
5:	learn: 11.6037509	total: 39.2ms	remaining: 6.49s
6:	learn: 10.9856432	total: 43.4ms	remaining: 6.15s
7:	learn: 10.4702333	total: 49ms	remaining: 6.08s
8:	learn: 10.0218410	total: 55.3ms	remaining: 6.09s
9:	learn: 9.6103004	total: 59.5ms	remaining: 5.89s
10:	learn: 9.2530414	total: 67ms	remaining: 6.03s
11:	learn: 8.9244833	total: 71.6ms	remaining: 5.89s
12:	learn: 8.6342228	total: 74.6ms	remaining: 5.66s
13:	learn: 8.3905163	total: 79.3ms	remaining: 5.59s
14:	learn: 8.1498552	total: 83.2ms	remaining: 5.46s
15:	learn: 7.9646248	total: 87ms	remaining: 5.35s
16:	learn: 7.7885656	total: 91.4ms	remaining: 5.29s
17:	learn: 7.5988991	total: 94.5ms	remaining: 5.15s
18:	learn: 7.4438594	total: 98ms	remaining: 5.06s
19:	learn: 7.2963209	t

In [ ]:
y_pred_test = model.predict(x_test)

In [ ]:
y_pred_test

array([34.47267083, 19.54777738, 10.77634482, ..., 41.39301212,
       46.24686127, 28.83117826])

In [ ]:
df_test_imputed['id'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 11049 entries, 0 to 11048
Series name: id
Non-Null Count  Dtype
--------------  -----
11049 non-null  int64
dtypes: int64(1)
memory usage: 86.4 KB


In [ ]:
submission_df = pd.DataFrame({'id': df_test_imputed['id'], 'Degerlendirme Puani': y_pred_test})
print(submission_df)


          id  Degerlendirme Puani
0          0            34.472671
1          1            19.547777
2          2            10.776345
3          3            16.620709
4          4            37.177097
...      ...                  ...
11044  11044            31.015842
11045  11045            31.864330
11046  11046            41.393012
11047  11047            46.246861
11048  11048            28.831178

[11049 rows x 2 columns]


In [ ]:
submission_df.head()

,id,Degerlendirme Puani
0,0,34.472671
1,1,19.547777
2,2,10.776345
3,3,16.620709
4,4,37.177097


In [ ]:
submission_df.to_csv('submission.csv', index=False)